In [ ]:
################################newidea

from mytoolkit2 import stepwise
import numpy as np

def cal_wqs(input_dir, thres, step):
    files = os.listdir(input_dir)
    wqs = []

    for f in files:
        array= np.loadtxt(os.path.join(input_dir, f))
        #array[array<0] = 0
       
        #array = stepwise.sparsity(array, thres=thres)
        array = stepwise.row_sparsity(array, thres=thres)
      
        wq, *_ = stepwise.findwalks_v2(array, step, normalize=stepwise.min_max)
        wqs.append(wq)
    return wqs

input_dir1 = r'E:\006_ET_MRgFUS\09_gradient_batch2\01_zform_data_group\ncnc'
input_dir2 = r'E:\006_ET_MRgFUS\09_gradient_batch2\01_zform_data_group\000d'
input_dir3 = r'E:\006_ET_MRgFUS\09_gradient_batch2\01_zform_data_group\180d'

wqs_ncnc = cal_wqs(input_dir1, 0.90, 120)
wqs_000d = cal_wqs(input_dir2, 0.90, 120)
wqs_180d = cal_wqs(input_dir3, 0.90, 120)

In [ ]:
import pandas as pd
subnetwork_path = r'E:\006_ET_MRgFUS\09_gradient_batch2\02_nii_mask\7network_atlas1039.csv'
subnetwork_df = pd.read_csv(subnetwork_path)
subnetwork_labels = subnetwork_df['Subnetwork'].values

subnetwork_labels[1025:1039] = 0

In [ ]:
import pandas as pd
from scipy.stats import ttest_rel,ttest_ind


def get_all_matrixs_stepi(stepi, wqs):
    matrixs = []
    for wq in wqs:
        matrixs.append(wq[stepi])
    return np.array(matrixs)

def get_subnetwork_mean(sub_col, subnetwork_labels):
    means = []
    for j in range(1, 8):
        subnetwork_value = sub_col[subnetwork_labels==j]
        mean = np.nansum(subnetwork_value)
        means.append(mean)
    return means

def get_subnetwork_means(matrixs, subnetwork_labels):
    subs_means = []
    for matrix in matrixs:
        cols = matrix[:, 1025:1039]
        means = get_subnetwork_mean(cols, subnetwork_labels)
        subs_means.append(means)
    return np.array(subs_means)

for axi in range(0, 100):
    stable_ncnc = get_all_matrixs_stepi(axi, wqs_ncnc)
    stable_000d = get_all_matrixs_stepi(axi, wqs_000d)
    stable_180d = get_all_matrixs_stepi(axi, wqs_180d)

    subs_means_ncnc = get_subnetwork_means(stable_ncnc, subnetwork_labels)
    subs_means_000d = get_subnetwork_means(stable_000d, subnetwork_labels)
    subs_means_180d = get_subnetwork_means(stable_180d, subnetwork_labels)

    ts, ps = ttest_rel(subs_means_180d, subs_means_000d)
    df = pd.DataFrame({'roi': range(1, np.shape(ts)[0]+1), 't_value': ts, 'p_value': ps})

    print("###########################")
    print("step", axi+1)  
    print(df)
    #df.to_csv(r'E:\006_ET_MRgFUS\14_SFC_analysis\1.csv')
    
    '''
    ##对于特定模块进行绘图
    col = 0 #模块1
    mean_values = subs_means_ncnc[:, col].tolist() + subs_means_000d[:, col].tolist() + subs_means_180d[:, col].tolist()
    x = [1 for _ in range(len(subs_means_ncnc))] + [2 for _ in range(len(subs_means_000d))] + [3 for _ in range(len(subs_means_180d))]
    
    sns.barplot(x=x, y=mean_values, palette='Set2')
    plt.show()
    plt.close()
    '''


In [ ]:
values = []
for i in range(np.shape(wqs_000d[0])[0]):
    wq = wqs_000d[2][i] #选择人
    cols = wq[:, 1025:1039]
    sub_matrix = cols[subnetwork_labels==2]
    value = np.sum(sub_matrix)
    values.append(value)
print(values)

values = []
for i in range(np.shape(wqs_000d[0])[0]):
    wq = wqs_000d[2][i] #选择人
    cols = wq[:, 1025:1039]
    sub_matrix = cols[subnetwork_labels==6]
    value = np.sum(sub_matrix)
    values.append(value)
print(values)

In [ ]:
import pandas as pd
from scipy.stats import ttest_rel,ttest_ind


def get_all_matrixs_stepi(stepi, wqs):
    matrixs = []
    for wq in wqs:
        matrixs.append(wq[stepi])
    return np.array(matrixs)

def get_subnetwork_mean(sub_col, subnetwork_labels):
    means = []
    for j in range(1, 8):
        subnetwork_value = sub_col[subnetwork_labels==j]
        mean = np.nansum(subnetwork_value)
        means.append(mean)
    return means

def get_subnetwork_means(matrixs, subnetwork_labels):
    subs_means = []
    for matrix in matrixs:
        cols = matrix[:, 1025:1039] #cols = matrix[:, 1025:1039]
        means = get_subnetwork_mean(cols, subnetwork_labels)
        subs_means.append(means)
    return np.array(subs_means)

def get_percent(values):
    total = np.nansum(values)
    return values/total

def get_subs_percent(subs_values):
    subs_percents = []
    for sub_values in subs_values:
        subs_percents.append(get_percent(sub_values))
    return np.array(subs_percents)

def acc_sum(subs_percent):
    for i in range(1, np.shape(subs_percent)[1]):
        subs_percent[:, i] = subs_percent[:, i] + subs_percent[:, i-1]
    return subs_percent


    


In [ ]:
from mytoolkit1.ttest import ind_cohen_d, rel_cohen_d

values1 = []
values1d = []
values2 = []
values2d = []
values3 = []
values3d = []
x = []
hue = []

for axi in range(1, 120):
    stable_ncnc = get_all_matrixs_stepi(axi, wqs_ncnc)
    stable_000d = get_all_matrixs_stepi(axi, wqs_000d)
    stable_180d = get_all_matrixs_stepi(axi, wqs_180d)

    subs_means_ncnc = get_subnetwork_means(stable_ncnc, subnetwork_labels)
    subs_means_000d = get_subnetwork_means(stable_000d, subnetwork_labels)
    subs_means_180d = get_subnetwork_means(stable_180d, subnetwork_labels)

    subs_percent_ncnc = get_subs_percent(subs_means_ncnc)
    subs_percent_000d = get_subs_percent(subs_means_000d)
    subs_percent_180d = get_subs_percent(subs_means_180d)

    ts1, ps1 = ttest_ind(subs_percent_000d, subs_percent_ncnc)
    d1 = ind_cohen_d(subs_percent_000d, subs_percent_ncnc)
    
    ts2, ps2 = ttest_rel(subs_percent_180d, subs_percent_000d)
    d2 = rel_cohen_d(subs_percent_180d, subs_percent_000d)

    ts3, ps3 = ttest_ind(subs_percent_180d, subs_percent_ncnc)
    d3 = ind_cohen_d(subs_percent_180d, subs_percent_ncnc)


    values1 += ts1.tolist()
    values1d += d1.tolist()
    values2 += ts2.tolist()
    values2d += d2.tolist()
    values3 += ts3.tolist()
    values3d += d3.tolist()

    x += [axi for i in range(7)]
    hue += [1,2,3,4,5,6,7]
    


In [ ]:
import seaborn as sns

values1 = np.array(values1)
values2 = np.array(values2)
values3 = np.array(values3)
x = np.array(x)
hue = np.array(hue)

values1_2 = values2[hue==2].tolist()
hue_2 = hue[hue==2].tolist()
x_2 = x[hue==2].tolist()

values1_6 = values2[hue==6].tolist()
hue_6 = hue[hue==6].tolist()
x_6 = x[hue==6].tolist()

new_values = values1_2 + values1_6
new_hue = hue_2 + hue_6
new_x = x_2 + x_6

color=["#ff861a","#895246"]

sns.set_theme(style="whitegrid", palette="pastel")
sns.relplot(y=new_values, hue=new_hue, x=new_x,kind='line',palette=color,  linewidth=4, height=6, aspect=1.5)
